<a href="https://colab.research.google.com/github/dmipatriot/fantasy_football_reports/blob/main/Survivor_Pool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install espn-api requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 4.2 MB/s eta 0:00:00


In [2]:
"""
Survivor Pool Tracker for ESPN Fantasy (Colab-friendly)

- Uses espn-api `League`
- Starts eliminating teams in Week 3 or whatever week is set in the settings
- Each week, the *lowest scoring* team that hasn't already been eliminated
  is knocked out of the survivor pool.
- Intended to be run manually once per week in Colab; it prints everything.
"""
from dataclasses import dataclass
from google.colab import userdata
from typing import Dict, List, Iterable, Tuple, Optional
from espn_api.football import League

# --------------------------------------------------
# CONFIG – reuse the same values from FF_Efficiency_Tracker
# --------------------------------------------------
LEAGUE_ID = userdata.get("LEAGUE_ID")
ESPN_S2 = userdata.get('ESPN_S2')
SWID = userdata.get('SWID')

# Survivor Settings
SURVIVOR_START_WEEK = 3   # first week to start eliminating teams
SEASON    = 2025


In [ ]:



# Optional: cap the latest week considered.
# - Set to an int (e.g., 8) if you want to lock in "through Week X".
# - Leave as None to use `league.current_week - 1` (last completed week).
MAX_WEEK: Optional[int] = None


@dataclass
class WeeklyScore:
    week: int
    team_id: int
    team_name: str
    points: float


def init_league() -> League:
    """
    Initialize the ESPN league object using the same cookies as your other notebook.
    """
    league = League(
        league_id=LEAGUE_ID,
        year=SEASON,
        espn_s2=ESPN_S2,
        swid=SWID,
    )
    return league


def get_week_range(league: League) -> List[int]:
    """
    Determine which weeks to include for the survivor calculation.
    """
    if MAX_WEEK is not None:
        last_week = MAX_WEEK
    else:
        # Use last *completed* week if possible
        try:
            # Many ESPN leagues expose `current_week` as the week in progress,
            # so use one less as the last completed week.
            last_week = int(getattr(league, "current_week", 18)) - 1
        except Exception:
            last_week = 18  # fallback, typical length of ESPN regular season

    if last_week < SURVIVOR_START_WEEK:
        raise ValueError(
            f"Last week ({last_week}) is before survivor start week ({SURVIVOR_START_WEEK}). "
            "Bump MAX_WEEK or wait until more weeks are played."
        )

    return list(range(SURVIVOR_START_WEEK, last_week + 1))


def fetch_weekly_scores(league: League, weeks: Iterable[int]) -> List[WeeklyScore]:
    """
    Pull weekly total scores for each team using box scores.
    """
    all_scores: List[WeeklyScore] = []

    # Build a lookup of team_id -> team_name once
    id_to_name: Dict[int, str] = {
        t.team_id: t.team_name for t in league.teams
    }

    for wk in weeks:
        print(f"[info] Fetching box scores for Week {wk}...")
        try:
            box_scores = league.box_scores(wk)
        except Exception as e:
            print(f"[warn] Could not fetch Week {wk} box scores: {e}")
            continue

        seen_ids = set()  # guard against duplicates just in case

        for box in box_scores:
            # Home team
            if box.home_team and box.home_team.team_id not in seen_ids:
                tid = box.home_team.team_id
                all_scores.append(
                    WeeklyScore(
                        week=wk,
                        team_id=tid,
                        team_name=id_to_name.get(tid, box.home_team.team_name),
                        points=float(box.home_score or 0.0),
                    )
                )
                seen_ids.add(tid)

            # Away team
            if box.away_team and box.away_team.team_id not in seen_ids:
                tid = box.away_team.team_id
                all_scores.append(
                    WeeklyScore(
                        week=wk,
                        team_id=tid,
                        team_name=id_to_name.get(tid, box.away_team.team_name),
                        points=float(box.away_score or 0.0),
                    )
                )
                seen_ids.add(tid)

    if not all_scores:
        print("[warn] No scores fetched – double-check cookies, season, or weeks.")
    return all_scores


def compute_survivor_eliminations(
    scores: List[WeeklyScore],
) -> Tuple[Dict[int, WeeklyScore], Dict[int, str]]:
    """
    Given per-week scores, compute survivor eliminations.

    Returns:
    - eliminated_by_week: {week -> WeeklyScore of eliminated team}
    - team_names: {team_id -> team_name}
    """
    if not scores:
        return {}, {}

    team_names: Dict[int, str] = {}
    by_week: Dict[int, List[WeeklyScore]] = {}

    for s in scores:
        team_names[s.team_id] = s.team_name
        by_week.setdefault(s.week, []).append(s)

    eliminated_by_week: Dict[int, WeeklyScore] = {}
    eliminated_team_ids: set[int] = set()

    for week in sorted(by_week.keys()):
        candidates = [
            s for s in by_week[week] if s.team_id not in eliminated_team_ids
        ]
        if not candidates:
            # All teams already eliminated (e.g., late playoff weeks)
            continue

        # Find the lowest score; break ties with team_id for determinism
        loser = sorted(candidates, key=lambda s: (s.points, s.team_id))[0]
        eliminated_by_week[week] = loser
        eliminated_team_ids.add(loser.team_id)

    return eliminated_by_week, team_names


def print_survivor_results(
    eliminated_by_week: Dict[int, WeeklyScore],
    team_names: Dict[int, str],
):
    """
    Pretty-print survivor eliminations and still-alive teams.
    """
    if not team_names:
        print("No data to show.")
        return

    eliminated_team_ids = {s.team_id for s in eliminated_by_week.values()}

    print("Eliminated:")
    for wk in sorted(eliminated_by_week.keys()):
        rec = eliminated_by_week[wk]
        print(f"Week {wk}: {rec.team_name} - {rec.points:.2f} points")

    print("\nStill Alive:")
    for tid, name in sorted(team_names.items(), key=lambda kv: kv[0]):
        if tid not in eliminated_team_ids:
            print(name)


def main():
    league = init_league()
    weeks = get_week_range(league)
    print(f"[info] Survivor weeks considered: {weeks[0]}–{weeks[-1]}")

    scores = fetch_weekly_scores(league, weeks)
    eliminated_by_week, team_names = compute_survivor_eliminations(scores)

    print()
    print_survivor_results(eliminated_by_week, team_names)


if __name__ == "__main__":
    main()
